<a href="https://colab.research.google.com/github/akanbiabiodun25/An-Analysis-of-Zero-Emission-Vehicles-registered-under-the-iZEV-program-under-Transport-Canada/blob/main/Hamoye_Stage_D_Model_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

# Save the checkpoint name (assuming you want to use a pre-trained model)
checkpoint = "bert-base-uncased"  # "bert-base-incased" is also a valid option

# Load tokenizer from the pre-trained model
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Load model from the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

# Sample sentences
sentences = ["I've been waiting for a HuggingFace course my whole life.", "This course is amazing!"]
labels = torch.tensor([1, 1])  # Assuming labels are 1 for both sentences

# Tokenize sentences with padding, truncation, and return tensors
batch = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

# Add "labels" to the batch dictionary (assuming it's for classification)
batch["labels"] = labels

# Create optimizer
optimizer = AdamW(model.parameters())  # Using AdamW optimizer

# Forward pass (assuming you want to classify the sentiment of the sentences)
outputs = model(**batch)  # Unpack the batch dictionary for the model
loss = outputs.loss  # Extract the loss from the model output

# Backward pass and parameter update
loss.backward()
optimizer.step()

print("Training complete (assuming one training step here)")  # Add for clarity

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch

Training complete (assuming one training step here)


In [ ]:
!pip install datasets
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")

print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})


In [ ]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [ ]:
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [ ]:
from transformers import AutoTokenizer

In [ ]:

checkpoint = "bert-base-uncased"  # Model identifier, not a path
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# Tokenize sentence 1 from the "sentence" column in the training split
tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])

# Tokenize sentence 2 from the "sentence" column in the training split
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])

In [ ]:
inputs = tokenizer("This is the first sentence", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:


from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"])
tokens

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 'sentence',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 'one',
 '.',
 '[SEP]']

In [ ]:

tokens = ['[CLS]', "this", "is", "the", "first", "sentence", ',', '[SEP]', "this", "is", "the","second", ',', '[SEP]']
attention_mask = [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]

# Use tokens and attention_mask separately for your model or further processing

In [ ]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding= True,
    truncation=True,
)

In [ ]:
def tokenize_function(example):


  return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Explanation about how Datasets library works
'''The way the 🤗 Datasets library applies this processing is by adding new fields to the datasets,
one for each key in the dictionary returned by the preprocessing function:'''

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

'The way the 🤗 Datasets library applies this processing is by adding new fields to the datasets,\none for each key in the dictionary returned by the preprocessing function:'

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

data_collator

DataCollatorWithPadding(tokenizer=BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [ ]:
samples = tokenized_datasets["train"][:8]

samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}

[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

In [ ]:

batch = data_collator(samples)

# Print the shapes of elements in the batch
print({k: v.shape for k, v in batch.items()})

{'input_ids': torch.Size([8, 67]), 'token_type_ids': torch.Size([8, 67]), 'attention_mask': torch.Size([8, 67]), 'labels': torch.Size([8])}


In [ ]:
from datasets import load_dataset  # Correct import

from transformers import AutoTokenizer, DataCollatorWithPadding

# Load the dataset
raw_datasets = load_dataset("glue", "mrpc")  # Consistent variable name

# Define the pre-trained model checkpoint
checkpoint = "bert-base-uncased"  # Typo correction: 'bert'

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)  # Typo correction: 'pretrained'

def tokenize_function(example):
    """Tokenizes a pair of sentences with truncation."""
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# Tokenize the dataset in batches
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Create the data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [ ]:
!pip install accelerate -U

!pip install transformers[torch]
from transformers import TrainingArguments
training_args = TrainingArguments("test-trainer")

In [ ]:
from transformers import AutoModelForSequenceClassification

# Define the pre-trained model checkpoint
checkpoint = "bert-base-uncased"  # Replace with your desired checkpoint name

# Load the pre-trained model (assuming num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import __version__
print(__version__)

4.41.1


In [ ]:
from transformers import AutoModelForSequenceClassification

# Define the pre-trained model checkpoint name (replace with your desired checkpoint)
checkpoint = "bert-base-uncased"

# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

from transformers import Trainer

trainer = Trainer(
    model,  # Replace with your loaded model
    training_args,  # Replace with your TrainingArguments instance
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

Step,Training Loss


KeyboardInterrupt: 

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])

print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:

!pip install evaluate


import evaluate

# Load the ERPC metric from the glue library
metric = evaluate.load("glue", "mrpc")  # Typo correction: "mrpc" for ERPC

# Prepare the ground truth data (assuming labels are in 'label_ids')
#references = predictions.label_ids

# Compute the metric scores
metric.compute(predictions=preds, references=predictions.label_ids)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.8 MB/s eta 0:00:00


{'accuracy': 0.31862745098039214, 'f1': 0.0071428571428571435}

In [ ]:
def compute_metrics(eval_preds):
    """
    Computes the MRPC metric on the provided evaluation predictions.

    Args:
        eval_preds (dict): A dictionary containing evaluation predictions.
            Expected keys: "logits" (model outputs before softmax) and "label_ids" (ground truth labels).

    Returns:
        dict: A dictionary containing the computed MRPC metric scores.
    """

    metric = evaluate.load("glue", "mrpc")  # Load the MRPC metric

    logits,labels = eval_preds["label_ids"]

    predictions = np.argmax(logits, axis=-1)  # Find predicted class labels

    return metric.compute(predictions=predictions, references=labels)

In [ ]:
!pip show transformers

Name: transformers
Version: 4.41.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


In [ ]:

training_args=TrainingArguments("test-trainer", evaluation_strategy="epoch")


# Model loading
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Create the Trainer instance
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # Use the defined `compute_metrics` function
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

# Load the MRPC dataset (assuming you want MRPC data)
raw_datasets = load_dataset("glue", "mrpc")

# Pre-trained checkpoint name
checkpoint = "bert-base-uncased"

# Load the tokenizer for the pre-trained model
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    """
    Tokenizes a single example from the dataset, handling both sentences for MRPC.

    Args:
        example (dict): A dictionary containing the data for a single example.
            Expected keys: "sentence1" (first sentence), "sentence2" (second sentence).

    Returns:
        dict: A dictionary containing the tokenized data with truncation if applicable.
    """

    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


# Apply the tokenize_function to all examples in the dataset (batched processing)
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Create the data collator for padding and batching
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# Remove unnecessary columns
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])

# Rename the label column (assuming "label" exists)
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

# Set the format to PyTorch tensors
tokenized_datasets.set_format("torch")

# Print column names for the training set
print(tokenized_datasets["train"].column_names)

In [ ]:

from torch.utils.data import DataLoader

# Train Data Loader
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator  # Use '=' for keyword assignment
)

# Evaluation Data Loader
eval_dataloader = DataLoader(
    tokenized_datasets["validation"],
    batch_size=8,
    collate_fn=data_collator  # Use '=' for keyword assignment
)

In [ ]:
# Get a single batch from the training data loader
batch = next(iter(train_dataloader))

# Print the shapes of all elements in the batch
print({k: v.shape for k, v in batch.items()})

In [ ]:
outputs = model(batch)

print(outputs.loss, outputs.logits.shape)

In [ ]:
from transformers import AdamW  # Use AdamW optimizer


# Create the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from transformers import get_scheduler

num_epochs = 3

# Calculate the total number of training steps
num_training_steps = num_epochs * len(train_dataloader)

# Define the learning rate scheduler
lr_scheduler = get_scheduler(
    "linear",  # Choose the desired scheduler type (e.g., "linear", "cosine")
    optimizer=optimizer,
    num_warmup_steps=0,  # Adjust warmup steps if needed
    num_training_steps=num_training_steps,
)

print(f"Number of training steps: {num_training_steps}")

In [ ]:
import torch

# Check for GPU availability
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Move the model to the chosen device
model.to(device)

# Print the device information (optional)
print(f"Using device: {device}")

In [ ]:
# Import libraries (assuming tqdm for progress bar)
from tqdm import tqdm

# Training loop
#num_training_steps = num_epochs * len(train_dataloader)
progress_bar = tqdm(range(num_training_steps))  # Initialize progress bar

model.train()  # Set model to training mode

for epoch in range(num_epochs):
    for batch in train_dataloader:
        # Move data to device (GPU/CPU)
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss  # Extract the loss from outputs

        # Backward pass
        loss.backward()

        # Optimizer steps
        optimizer.step()
        lr_scheduler.step()

        # Zero gradients
        optimizer.zero_grad()

        # Update progress bar (assuming one update per batch)
        progress_bar.update(1)

# Close progress bar
#progress_bar.close()

In [ ]:
import evaluate

# Assuming the metric is accuracy (replace with your desired metric)
metric = evaluate.load("glue", "mrpc")

# Evaluation loop
model.eval()  # Set model to evaluation mode

for batch in eval_dataloader:
    # Move data to device (GPU/CPU)
    batch = {k: v.to(device) for k, v in batch.items()}

    # Forward pass with gradient suppression
    with torch.no_grad():
        outputs = model(batch)
        logits = outputs.logits  # Extract logits from outputs

    # Predictions and metric calculation
    predictions = torch.argmax(logits, dim=-1)  # Corrected dimension for argmax
    metric.add_batch(predictions=predictions, references=batch["labels"])

# Compute the overall metric score
metric.compute()

In [ ]:
linear = "cosine"  # Assuming linear refers to the scheduler type

# Learning rate scheduler
lr_scheduler = get_scheduler(
    linear,
    optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:



from transformers import AutoModelForSequenceClassification, AdamW, get_scheduler

# Create model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# Optimizer and learning rate
optimizer = AdamW(model.parameters(), lr=3e-5)  # Adjust learning rate as needed

# Device selection (GPU or CPU)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Training parameters
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)

# Learning rate scheduler (optional)
# Consider using a scheduler like CosineAnnealingLR or ReduceLROnPlateau
lr_scheduler = get_scheduler(
linear,
optimizer,
num_warmup_steps=0,
num_training_steps = num_training_steps,
)

from tqdm import tqdm  # Import tqdm for progress bar

# Training loop
progress_bar = tqdm(range(num_training_steps))  # Initialize progress bar

model.train()  # Set model to training mode

for epoch in range(num_epochs):
    for batch in train_dataloader:
        # Move data to device
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(batch)
        loss = outputs.loss

        # Backward pass
        loss.backward()

        # Optimizer steps
        optimizer.step()

        # Learning rate scheduler (if used)
        #if lr_scheduler is not None:
        lr_scheduler.step()

        # Zero gradients
        optimizer.zero_grad()

        # Progress bar update
        progress_bar.update(1)

# Close progress bar
#progress_bar.close()

In [ ]:
from accelerate import Accelerator
from transformers import AutoModelForSequenceClassification, AdamW, get_scheduler

# Initialize Accelerator (adjust device configuration if needed)
accelerator = Accelerator()

# Load pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# Create optimizer
optimizer = AdamW(model.parameters(), lr=3e-5)  # Adjust learning rate as needed

# Prepare datasets and model on accelerator
train_dl, eval_dl, model, optimizer = accelerator.prepare(train_dataloader, eval_dataloader,
model, optimizer)

# Training parameters
num_epochs = 3
num_training_steps = num_epochs * len(train_dl)

# Learning rate scheduler (optional)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,  # Adjust warmup steps if needed
    num_training_steps=num_training_steps,
)

# Progress bar (optional)
from tqdm import tqdm
progress_bar = tqdm(range(num_training_steps))  # Uncomment to use a progress bar

model.train()  # Set model to training mode

for epoch in range(num_epochs):
    for batch in train_dl:
        # Move data to device (handled by Accelerator)
        outputs = model(**batch)
        loss = outputs.loss

        # Backward pass
        accelerator.backward(loss)

        # Optimizer steps
        optimizer.step()

        # Learning rate scheduler (if used)
        #if lr_scheduler is not None:
        lr_scheduler.step()

        # Zero gradients
        optimizer.zero_grad()

        # Progress bar update (if used)
        #if progress_bar is not None:
        progress_bar.update(1)

# Close progress bar (if used)
#if progress_bar is not None:
    #progress_bar.close()

#print("Training complete!")